In [ ]:
import board
import time
import busio
import digitalio
spi = busio.SPI(board.SCK, board.MOSI, board.MISO)

while not spi.try_lock():
    pass
spi.configure(baudrate=50000, phase=0, polarity=0)
result = bytearray(1)
spi.write([0x80])
print(spi.readinto(result))


def read_from_register(address, num_bytes):
    read_command = bytearray([address])
    #CS.value = False  # Assert CS to low to start the transaction
    spi.write(read_command)
    time.sleep(0.00000002)  # Small delay
    time.sleep(0.00000002)  # Small delay
    time.sleep(0.00000002)  # Small delay
    time.sleep(0.00000002)  # Small delay
    time.sleep(0.00000002)  # Small delay
    received_data = bytearray(num_bytes)
    spi.readinto(received_data)
    #CS.value = True  # Deassert CS to high to end the transaction
    return received_data
spi.unlock()

In [ ]:
import board
import digitalio
import busio
import time

# Define SPI clock speed (adjust based on your device requirements)
SPI_CLOCK_SPEED = 4000000 # Adjust based on your device requirements (in Hz)

# Configure SPI pins (modify based on your FT232H breakout board layout)
SCK = board.SCK
MOSI = board.MOSI
MISO = board.MISO
CS = digitalio.DigitalInOut(board.D5)  # Replace D5 with the actual CS pin

CS.direction = digitalio.Direction.OUTPUT

# Initialize SPI bus
spi = busio.SPI(SCK, MOSI, MISO)

# Try to lock the SPI bus
while not spi.try_lock():
    pass

# Configure SPI clock frequency
spi.configure(baudrate=SPI_CLOCK_SPEED)

def write_to_register(address, data):
    command = bytearray([address, data])
    CS.value = False  # Assert CS to low to start the transaction
    spi.write(command)
    CS.value = True  # Deassert CS to high to end the transaction

def read_from_register(address, num_bytes):
    read_command = bytearray([address])
    CS.value = False  # Assert CS to low to start the transaction
    spi.write(read_command)
    time.sleep(0.00000002)  # Small delay
    time.sleep(0.00000002)  # Small delay
    time.sleep(0.00000002)  # Small delay
    time.sleep(0.00000002)  # Small delay
    time.sleep(0.00000002)  # Small delay
    received_data = bytearray(num_bytes)
    spi.readinto(received_data)
    CS.value = True  # Deassert CS to high to end the transaction
    return received_data

def format_as_binary(data):
    binary_string = ''.join(f'{byte:08b}' for byte in data)
    return binary_string

# Example usage (modify addresses and data values based on your device)
CS.value = True
time.sleep(2)



time.sleep(2)
read_data = read_from_register(0x80, 2)  # Read 2 bytes from register 0x00

# Format and print the read data as binary
binary_output = format_as_binary(read_data)
print("Read data (binary):", binary_output)

time.sleep(2)  # Add a delay if needed between operations

# Release the SPI bus
spi.unlock()  # Unlock the SPI bus
spi.deinit()  # Clean up resources when done

In [ ]:
import busio
import digitalio
from board import *
from adafruit_bus_device.spi_device import SPIDevice
with busio.SPI(SCK, MOSI, MISO) as spi_bus:
    #cs = digitalio.DigitalInOut(D10)
    device = SPIDevice(spi_bus)
    bytes_read = bytearray(10)
    # The object assigned to spi in the with statements below
    # is the original spi_bus object. We are using the busio.SPI
    # operations busio.SPI.readinto() and busio.SPI.write().
    with device as spi:
        spi.write(b'x81')
    with device as spi:
        print(spi.readinto(bytes_read))
    # A second transaction
    
        

In [ ]:
# See https://blog.3d-logic.com/2015/01/10/using-a-tm1638-based-board-with-arduino/
import atexit

from pyftdi.spi import SpiController
from time import sleep

SPI_FREQ = 10000
SPI_MODE = 0

# Initialize the controller
spi = SpiController()
spi.configure("ftdi://ftdi:232h/1")

In [ ]:
# See https://blog.3d-logic.com/2015/01/10/using-a-tm1638-based-board-with-arduino/
import atexit

from pyftdi.spi import SpiController
from time import sleep

SPI_FREQ = 10000
SPI_MODE = 0

# Initialize the controller
spi = SpiController()
spi.configure("ftdi://ftdi:232h/1")

# This function reverses the bits in a byte. This is a quirk of this device,
# paired with the FT232H, you need to swap endianness.
# Note that this is a crappy way to do this and Python in theory has better
# ways to work with raw bytes, but it's not working right for me and I don't
# want to figure it out right now.
def rev(x):
    x = ("{:08b}".format(x))[::-1]
    return int(x, base=2)

# Register a callback to relinquish control of the FT232H when the program exits.
atexit.register(spi.terminate)

# Get a port to the slave with the appropriate chip select line.
slave = spi.get_port(cs=0, freq=SPI_FREQ, mode=SPI_MODE) # 0 = d3

###################################

### Addresses for the LEDs and segments, see the datasheet.
# Indexes for ADDR_DIGITS and ADDR_LEDS match the positions on the board,
# from left to right.
ADDR_ALL = [(0xc0 + i) for i in range(16)]
ADDR_DIGITS = ADDR_ALL[::2]
ADDR_LEDS = ADDR_ALL[1::2]


print ('[{}]'.format(', '.join(hex(x) for x in ADDR_LEDS)))

###################################

# Turn the device on and setting brightness to max 
# 0x8f = 0b10001111 = 0b10000000 & 0b00001000 & 0b00000111
#                     address      state = on   bright = 7
slave.write([rev(0x8f)])                                

### Initialize all segments to off
# First, turn on auto-increment mode, in this mode you write a starting address
# followed by up to 16 bytes, corresponding to each segment/LED.
# The device automatically rolls through the segments and assigns their
# values, starting from the first address and wrapping around.
# You don't HAVE to write 16 bytes, you can do fewer (I think).
slave.write([rev(0x40)]) # Increment mode
slave.write([rev(0xc0)] + 16 * [rev(0x00)]) # Write all zeros = all off

### Now write a message
# Bitmasks correspond to individual segments... 0bABCDEFGP, where A-G
# are the standard names of the segments in a 7-segment display and P is
# the period. 
#         -A-
#       |     |
#       F     B
#       |     |
#         -G-
#       |     |
#       E     C
#       |     |
#         -D-   P
# The items alternate between segments and LEDs. LEDs only use the
# least significant bit.
# 
# NOTE: I am not reversing the bitmasks! I'm just doing them in the
# correct order already because it's easier to think about.
slave.write([rev(0x40)]) # Increment mode

if True:
    slave.write([
        rev(0xC0),  # Start address
        0b00101010, # D1 N
        0b00000000, # L1
        0b10011110, # D2 E
        0b00000000, # L2
        0b00111000, # D3 U
        0b00000000, # L3
        0b00011110, # D4 T
        0b00000000, # L4
        0b00001010, # D5 R
        0b00000000, # L5
        0b11111100, # D6 O
        0b00000000, # L6
        0b00101010, # D7 N
        0b00000000, # L7
        0b10110111, # D8 S
        0b00000000, # L8
    ])

# Now I will make the LEDs blink back and forth to demonstrate address
# mode, where we write to one item at a time.

# state is just a list of bools to make it easier to toggle back and forth.
# Each item in the list will be used to set the state of the matching LED.
state = 4 * [False, True]

while False:
    for i, x in enumerate(state):
        addr = ADDR_LEDS[i] # Get the corresponding LED address
        val = 0x01 if x else 0x00 # LED on if state is true, false otherwise

        slave.write([rev(0x44)]) # Activate address mode
            
        # Write (address, val) to set the state of the specific LED
        slave.write([
            rev(addr),
            rev(val),
        ]) 

        # Flip the variable in state so that LEDs toggle back and forth
        state[i] = not x

    sleep(1)

### Extra lesson: reading buttons!
# The TM1638 also has onboard buttons that you can read the state of.
# Unfortunately, the chip actually multiplexes MISO and MOSI on the same
# line. This doesn't work very well with most hardware SPI implementations.
# It's possible to make it work with the FT232H, but is more trouble than
# it's worth and I'm just gonna show you how for demonstration purposes
# as if it did work.
#
# This requires a transaction, with a write followed by a read. You write
# 0x42, then read back 4 bytes. These bytes represent scancodes for a key
# matrix... the details aren't that important. Tl;dr is you need to OR
# the 4 bytes together to get a status of the keys.
# 
# from operator import or_ as bit_or
# from functools import reduce
#
## Write 0x42, then read back 4 bytes
# scan_codes = slave.exchange([rev(0x42)], readlen=4)
#
# key_mask = reduce(bit_or, scan_codes) # OR everything together
# print("0b{:08b}".format(key_mask)) 

In [ ]:
valx = 0b00000000
slave.write([rev(0x44)]) # Activate address mode
slave.write([rev(0xC1),
    valx]), # D1
slave.write([rev(0xC3),
    valx]), # D1
slave.write([rev(0xC5),
    valx]), # D1
slave.write([rev(0xC7),
    valx]), # D1
slave.write([rev(0xC9),
    valx]), # D1
slave.write([rev(0xCB),
    valx]), # D1
slave.write([rev(0xCD),
    valx]), # D1
slave.write([rev(0xCf),
    valx]), # D1
    

In [ ]:
slave.write([rev(0x8f)])    

In [ ]:
from operator import or_ as bit_or
from functools import reduce

#Write 0x42, then read back 4 bytes
#scan_codes = slave.exchange(cs_prolog =[rev(0x42)], readlen=8 , duplex=False)
#scan_codes = slave.exchange([rev(0x42)], readlen=8 )
slave.write([rev(0x42)])
scan_codes=slave.read(2)
print(scan_codes)
key_mask = reduce(bit_or, scan_codes) # OR everything together
print("0b{:08b}".format(key_mask))

In [1]:
"""

LED&KEY - TM1638 7-segment LED python display driver with keyscan
https://github.com/mcauser/micropython-tm1638

MIT License
Copyright (c) 2018 Mike Causer

come from :
MicroPython TM1638 7-segment LED display driver with keyscan
https://github.com/mcauser/micropython-tm1638

MIT License
Copyright (c) 2018 Mike Causer

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

# LED&KEY module features:
# 8x 7-segment decimal LED modules
# 8x individual LEDs
# 8x push buttons


import time
import board
import digitalio



TM1638_CMD   = 0x40  # 0x40 data command
TM1638_ADDR   = 0xC0 # 0xC0 address command
TM1638_DISPL   = 0x80 # 0x80 display control command
TM1638_DSP_ON = 0x08 # 0x08 display on
TM1638_READ   = 0x02    # 0x02 read key scan data
TM1638_FIXED  = 0x04  # 0x04 fixed address mode

# 0-9, a-z, blank, dash, star
_SEGMENTS = bytearray(b'\x3F\x06\x5B\x4F\x66\x6D\x7D\x07\x7F\x6F\x77\x7C\x39\x5E\x79\x71\x3D\x76\x06\x1E\x76\x38\x55\x54\x3F\x73\x67\x50\x6D\x78\x3E\x1C\x2A\x76\x6E\x5B\x00\x40\x63')

def rev(x):
    x = ("{:08b}".format(x))[::-1]
    return int(x, base=2)

def sleep_ms(x):
    time.sleep(x/1000)

def sleep_µs(x):
    time.sleep(x/1000000)

def binary(num, length=8):
    return format(num, '#0{}b'.format(length + 2))

class TM1638(object):
    """Library for the TM1638 LED display driver."""
    def __init__(self, brightness=7):

        self.stb = digitalio.DigitalInOut(board.C0)
        self.stb.direction = digitalio.Direction.OUTPUT
        
        self.clk = digitalio.DigitalInOut(board.C1)
        self.clk.direction = digitalio.Direction.OUTPUT

        self.dio = digitalio.DigitalInOut(board.C2)
        self.dio.direction = digitalio.Direction.OUTPUT

        if not 0 <= brightness <= 7:
            raise ValueError("Brightness out of range")
        self._brightness = brightness

        self._on = TM1638_DSP_ON

        #self.clk.init(Pin.OUT, value=1)
        #self.dio.init(Pin.OUT, value=0)
        #self.stb.init(Pin.OUT, value=1)
        
        self.clk.value = True
        self.dio.value = False
        self.stb.value = True

        #self.clear()
        #self._write_dsp_ctrl()

    def _write_data_cmd(self):
        # data command: automatic address increment, normal mode
        self._command(TM1638_CMD)

    def _set_address(self, addr=0):
        # address command: move to address
        self._byte(TM1638_ADDR | addr)

    def _write_dsp_ctrl(self):
        # display command: display on, set brightness
        self._command(TM1638_DISPL | self._on | self._brightness)

    def start_transmission(self):
        self.stb.value = False

        #print("STB 00000000000000")

    def end_transmission(self):

        self.stb.value = True
        
        #print("STB 11111111111111")

    def send_command(self,cmd):
        self._byte(cmd)

    def send_data(self,data):
        self._byte(data)
        

    def _command(self, cmd):
         self._byte(cmd)

    def _byte(self, b):
        
        #print(hex(b))
        #print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        for i in range(8):
            
            self.clk.value = False
            
            x = True if ((b >> i) & 1) else False # LED on if state is true, false otherwise
            self.dio.value = x
 
            self.clk.value = True
            
        #print("------------------------------------------------------------------")
        
    def _scan_keys(self):
        """Reads one of the four bytes representing which keys are pressed."""
        pressed = 0
        #self.dio.init(Pin.IN, Pin.PULL_UP)
        self.dio = digitalio.DigitalInOut(board.C2)
        self.dio.direction = digitalio.Direction.INPUT
        for i in range(8):
            self.clk.value = False
            sleep_µs(100)
            v_bool = self.dio.value
            if self.dio.value:
                pressed |= 1 << i
            self.clk.value = True
            sleep_µs(100)
        #self.dio.init(Pin.OUT)
        self.dio.direction = digitalio.Direction.OUTPUT
        return pressed

    def power(self, val=None):
        """Power up, power down or check status"""
        if val is None:
            return self._on == TM1638_DSP_ON
        self._on = TM1638_DSP_ON if val else 0
        self._write_dsp_ctrl()

    def brightness(self, val=None):
        """Set the display brightness 0-7."""
        # brightness 0 = 1/16th pulse width
        # brightness 7 = 14/16th pulse width
        if val is None:
            return self._brightness
        if not 0 <= val <= 7:
            raise ValueError("Brightness out of range")
        self._brightness = val
        self._write_dsp_ctrl()

    def clear(self):
        """Write zeros to each address"""
        self._write_data_cmd()
        self.stb.value = False
        self._set_address(0)
        for i in range(16):
            self._byte(0x00)
        self.stb.value = True

    def write(self, data, pos=0):
        """Write to all 16 addresses from a given position.
        Order is left to right, 1st segment, 1st LED, 2nd segment, 2nd LED etc."""
        if not 0 <= pos <= 15:
            raise ValueError("Position out of range")
        self._write_data_cmd()
        self.stb.value = False
        self._set_address(pos)
        for b in data:
            self._byte(b)
        self.stb.value = True

    def led(self, pos, val):
        """Set the value of a single LED"""
        self.write([val], (pos << 1) + 1)

    def leds(self, val):
        """Set all LEDs at once. LSB is left most LED.
        Only writes to the LED positions (every 2nd starting from 1)"""
        self._write_data_cmd()
        pos = 1
        for i in range(8):
            self.stb.value = False
            self._set_address(pos)
            self._byte((val >> i) & 1)
            pos += 2
            self.stb.value = True

    def segments(self, segments, pos=0):
        """Set one or more segments at a relative position.
        Only writes to the segment positions (every 2nd starting from 0)"""
        if not 0 <= pos <= 7:
            raise ValueError("Position out of range")
        self._write_data_cmd()
        for seg in segments:
            self.stb.value = False
            self._set_address(pos << 1)
            self._byte(seg)
            pos += 1
            self.stb.value = True

    def keys(self):
        """Return a byte representing which keys are pressed. LSB is SW1"""
        keys = 0
        self.stb.value = False
        self._byte(TM1638_CMD | TM1638_READ)
        for i in range(4):
            keys |= self._scan_keys() << i
        self.stb.value = True
        return keys

    def qyf_keys(self):
        """Return a 16-bit value representing which keys are pressed. LSB is SW1"""
        keys = 0
        self.stb.value = False
        self._byte(TM1638_CMD | TM1638_READ)
        for i in range(4):
            i_keys = self._scan_keys()
            for k in range(2):
                for j in range(2):
                    x = (0x04 >> k) << j*4
                    if i_keys & x == x:
                        keys |= (1 << (j + k*8 + 2*i))
        self.stb.value = True
        return keys

    def encode_digit(self, digit):
        """Convert a character 0-9, a-f to a segment."""
        return _SEGMENTS[digit & 0x0f]

    def encode_string(self, string):
        """Convert an up to 8 character length string containing 0-9, a-z,
        space, dash, star to an array of segments, matching the length of the
        source string excluding dots, which are merged with previous char."""
        segments = bytearray(len(string.replace('.','')))
        j = 0
        for i in range(len(string)):
            if string[i] == '.' and j > 0:
                segments[j-1] |= (1 << 7)
                continue
            segments[j] = self.encode_char(string[i])
            j += 1
        return segments

    def encode_char(self, char):
        """Convert a character 0-9, a-z, space, dash or star to a segment."""
        o = ord(char)
        if o == 32:
            return _SEGMENTS[36] # space
        if o == 42:
            return _SEGMENTS[38] # star/degrees
        if o == 45:
            return _SEGMENTS[37] # dash
        if o >= 65 and o <= 90:
            return _SEGMENTS[o-55] # uppercase A-Z
        if o >= 97 and o <= 122:
            return _SEGMENTS[o-87] # lowercase a-z
        if o >= 48 and o <= 57:
            return _SEGMENTS[o-48] # 0-9
        raise ValueError("Character out of range: {:d} '{:s}'".format(o, chr(o)))

    def hex(self, val):
        """Display a hex value 0x00000000 through 0xffffffff, right aligned, leading zeros."""
        string = '{:08x}'.format(val & 0xffffffff)
        self.segments(self.encode_string(string))

    def number(self, num):
        """Display a numeric value -9999999 through 99999999, right aligned."""
        # limit to range -9999999 to 99999999
        num = max(-9999999, min(num, 99999999))
        string = '{0: >8d}'.format(num)
        self.segments(self.encode_string(string))

    #def float(self, num):
    #    # needs more work
    #    string = '{0:>9f}'.format(num)
    #    self.segments(self.encode_string(string[0:9]))

    def temperature(self, num, pos=0):
        """Displays 2 digit temperature followed by degrees C"""
        if num < -9:
            self.show('lo', pos) # low
        elif num > 99:
            self.show('hi', pos) # high
        else:
            string = '{0: >2d}'.format(num)
            self.segments(self.encode_string(string), pos)
        self.show('*C', pos + 2) # degrees C

    def humidity(self, num, pos=4):
        """Displays 2 digit humidity followed by RH"""
        if num < -9:
            self.show('lo', pos) # low
        elif num > 99:
            self.show('hi', pos) # high
        else:
            string = '{0: >2d}'.format(num)
            self.segments(self.encode_string(string), pos)
        self.show('rh', pos + 2) # relative humidity

    def show(self, string, pos=0):
        """Displays a string"""
        segments = self.encode_string(string)
        self.segments(segments[:8], pos)

    def scroll(self, string, delay=250):
        """Display a string, scrolling from the right to left, speed adjustable.
        String starts off-screen right and scrolls until off-screen left."""
        segments = string if isinstance(string, list) else self.encode_string(string)
        data = [0] * 16
        data[8:0] = list(segments)
        for i in range(len(segments) + 9):
            self.segments(data[0+i:8+i])
            sleep_ms(delay)


In [3]:

tm = TM1638()

tm.clear()

tm.start_transmission()
tm._command((0x8f))
tm.end_transmission()

tm.start_transmission()
tm._command(TM1638_CMD | 0x04)
tm.end_transmission()

key = tm.keys()
print(binary(key))

tm.show("Bernard")

while True:
    
    key = tm.keys()
    sleep_µs(800)
    for i in range (1,16,2):
        #print(i,end=" shift ")
        shift = int((i-1)/2)
    
        #print(shift,end=" avec ")
        #print(binary((key >> shift)  & 1))
        x = True if (key >> shift & 1) else False # LED on if state is true, false otherwise
        tm.start_transmission()
        if x:
            tm._command(TM1638_ADDR | (i))
            tm._byte(0b00000001)
        else :
            tm._command(TM1638_ADDR | (i))
            tm._byte(0b00000000)
        tm.end_transmission() 



0b00000000


FtdiError: UsbError: [Errno 10060] Operation timed out

In [ ]:
import time
import board
import digitalio

stb = digitalio.DigitalInOut(board.C0)
stb.direction = digitalio.Direction.OUTPUT

clk = digitalio.DigitalInOut(board.C1)
clk.direction = digitalio.Direction.OUTPUT

dio = digitalio.DigitalInOut(board.C2)
dio.direction = digitalio.Direction.OUTPUT

def clear():
    """Write zeros to each address"""
    _command(0x40)
    stb.value = False
    _command(0xC0)
    for i in range(16):
        _byte(0x00)
    stb.value = True

def rev(x):
    x = ("{:08b}".format(x))[::-1]
    return int(x, base=2)

def sleep_ms(x):
    time.sleep(x/1000)

def sleep_µs(x):
    time.sleep(x/1000000)

def _command(cmd):
    _byte(cmd)

def _byte(b):
    

    for i in range(8):
        clk.value = False
        sleep_µs(2)

        x = True if ((b >> i) & 1) else False # LED on if state is true, false otherwise
        dio.value = x
        
        
        clk.value = True
        sleep_µs(2)


def start_transmission():
    stb.value = False
    sleep_µs(4)

    #print("STB 00000000000000")

def end_transmission():
    sleep_µs(2)
    stb.value = True
    
    
    #print("STB 11111111111111")
        

clear()

start_transmission()
_byte

start_transmission()
_command((0x44))
end_transmission()((0x8f))
end_transmission()

start_transmission()
_command(0xC1)
_byte(0b00000001)
end_transmission()

start_transmission()
_command(0xC3)
_byte(0b00000001)
end_transmission()

start_transmission()
_command(0xC5)
_byte(0b00000001)
end_transmission()


In [ ]:
b=0b0000011
print(bin(b << 5 & 0x80))
for i in range(8):
    print

In [ ]:

tm = TM1638()
while True:
    pressed = keys()
    print(pressed)
    for i in range(8):
        led(i, (pressed >> i) & 1)
        
    time.sleep(10/1000)

In [ ]:
# Wemos D1 Mini

tm = TM1638()


# turn them on, one by one
for i in range(8):
    led(i, 1)
    sleep_ms(200)

# turn them off, one by one
for i in range(8):
    led(i, 0)
    sleep_ms(200)

# toggle 1st LED
led(0, 1)
led(0, 0)

# toggle the 2nd LED
led(1, True)
led(1, False)

# turn on only the 4th LED
leds(8)

# turn on only the first 3 LEDs
leds(7)

# every 2nd led on
for i in range(10):
    leds(0b01010101)
    sleep_ms(100)
    leds(0b10101010)
    sleep_ms(100)

# leds off
leds(0)

In [ ]:
# Wemos D1 Mini
import tm1638
from machine import Pin
tm = tm1638.TM1638(stb=Pin(13), clk=Pin(14), dio=Pin(12))

from time import sleep_ms

# press a button to illuminate the matching individual LED
# you can press multiple buttons simultaneously
while True:
    pressed = keys()
    for i in range(8):
        led(i, (pressed >> i) & 1)
    sleep_ms(10)

In [ ]:
x=1
val = 0x01 if x else 0x00 # LED on if state is true, false otherwise
print(val)

In [ ]:
import board
import digitalio
button = digitalio.DigitalInOut(board.C0)
button.direction = digitalio.Direction.OUTPUT
button.value = False